In [19]:
!git clone https://github.com/yk03studios-wq/AnimeGANv3-fixed.git
%cd AnimeGANv3-fixed

In [20]:
!pip install -q onnxruntime-gpu tqdm Pillow
!pip install -q -r requirements.txt

In [21]:
!sed -i '/onnxruntime==/d' requirements.txt
!sed -i '/onnxruntime/d' requirements.txt
!sed -i '/tensorflow/d' requirements.txt

In [22]:
!pip install -q onnxruntime==1.22.1

In [23]:
!pip install -q -r requirements.txt

In [24]:
import onnxruntime as ort
print("ONNX Runtime:", ort.__version__)
print("Providers:", ort.get_available_providers())

In [ ]:
!python tools/video2anime.py -i inputs/vid/1.mp4 -o output/results -m deploy/AnimeGANv3_Hayao_36.onnx  

In [26]:
!ls -la output/results/

In [ ]:
import json
import subprocess
from pathlib import Path

# Possible output locations
CANDIDATE_OUTPUT_DIRS = [
    Path("output/results"),
    Path("/kaggle/working/output/results"),
]

OUTPUT_DIR = next((p for p in CANDIDATE_OUTPUT_DIRS if p.exists()), None)

if OUTPUT_DIR is None:
    raise RuntimeError(
        "Output directory not found. Checked:\n" +
        "\n".join(str(p) for p in CANDIDATE_OUTPUT_DIRS)
    )

print(f"✅ Found output directory: {OUTPUT_DIR.resolve()}")

DATASET_DIR = Path("/kaggle/working/dataset_export")
DATASET_DIR.mkdir(parents=True, exist_ok=True)

DATASET_OWNER = "yashkawade03"
DATASET_SLUG = "animegan-outputs-latest"
DATASET_ID = f"{DATASET_OWNER}/{DATASET_SLUG}"

# Copy outputs safely
for file in OUTPUT_DIR.iterdir():
    if file.is_file():
        target = DATASET_DIR / file.name
        target.write_bytes(file.read_bytes())

print(f"📦 Copied outputs to {DATASET_DIR}")

# Write dataset metadata
dataset_metadata = {
    "title": "AnimeGAN Outputs (Latest)",
    "id": DATASET_ID,
    "licenses": [{"name": "CC0-1.0"}]
}

metadata_path = DATASET_DIR / "dataset-metadata.json"
metadata_path.write_text(json.dumps(dataset_metadata, indent=2))

print("📝 dataset-metadata.json written")

# Create or update dataset (idempotent)
# --- Ensure Kaggle CLI auth inside notebook ---
import os
from pathlib import Path

kaggle_dir = Path("/root/.config/kaggle")
kaggle_dir.mkdir(parents=True, exist_ok=True)

kaggle_json = kaggle_dir / "kaggle.json"

if not kaggle_json.exists():
    kaggle_json.write_text(
        json.dumps(
            {
                "username": os.environ.get("KAGGLE_USERNAME"),
                "key": os.environ.get("KAGGLE_KEY"),
            }
        )
    )
    os.chmod(kaggle_json, 0o600)

print("🔐 Kaggle CLI authenticated")

cmd = [
    "kaggle", "datasets", "create",
    "-p", str(DATASET_DIR),
    "--dir-mode", "zip",
    "--force"
]

print("🚀 Running:", " ".join(cmd))
subprocess.run(cmd, check=True)

print(f"✅ Dataset updated successfully:")
print(f"👉 https://www.kaggle.com/datasets/{DATASET_ID}")